In [1]:
import requests
import pandas as pd
import defs
from datetime import datetime, timedelta
import pytz
import defs
import plotly.graph_objects as go 

/Users/egrados/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Debo obtener la data de ESTA semana. Parametros en RFC3339.  
Considerando que solo obtengo 5000 velas por llamada de la API y que cada dia tengo 1440 velasde M1, tengo que hacer 2.016 llamadas de la API por cada toda una semana.  
**NO HAY CONVERSION DE HORAS, LA HORA SOLO ES UNA Y VAMOS A USAR FORMATO Z UTC+0**  
- FORMATO RFC 3339:
    - 2024-07-03T03:08:00.000000000Z  (le resto 4 horas al UTC+0)


Como el API usa los valores Z UTC+0, voy a basar toda mi logica en este formato, bajo esa premisa mis horas importantes son:

- Inicio del dia de NY en formato Z: 04:00:00.000000000Z:
    - 2024-07-03T04:00:00.000000000Z
- Timeframe para Liquidity Grab NY 09:30-10:00 am:
    -   2024-07-03T13:30:00.000000000Z
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z

*NOTA:*  
Notar que en RFC339, la variable Z se usa para indicar que es UTC+0, por lo que la usare siempre (https://medium.easyread.co/understanding-about-rfc-3339-for-datetime-formatting-in-software-engineering-940aa5d5f68a).  


In [2]:
# Validacion de horario de verano en NY (retorno booleano) pero del dia de HOY.
import pytz
from datetime import datetime

def esta_en_horario_verano_nueva_york():
    # Definir la zona horaria de Nueva York
    zona_horaria_ny = pytz.timezone('America/New_York')

    # Obtener la fecha y hora actual en la zona horaria de Nueva York
    ahora_ny = datetime.now(zona_horaria_ny)

    # Verificar si estamos en horario de verano
    esta_en_horario_verano = ahora_ny.dst() != zona_horaria_ny.dst(None)

    return esta_en_horario_verano

esta_en_horario_verano_nueva_york()

True

In [3]:
# Obtengo la hora 04:00 de hace 7 dias, validando previamente si es horario de verano 
import pytz
from datetime import datetime, timedelta

def obtener_hora_04_00_utc_rfc3339_hace_7_dias():
    # Obtener la fecha y hora actual en UTC+0
    ahora_utc = datetime.now(pytz.utc)

    # Restar 7 días a la fecha y hora actual
    hace_7_dias = ahora_utc - timedelta(days=7)

    # Validar si es horario de verano
    horario_verano = esta_en_horario_verano_nueva_york()
    
    if horario_verano: 
        # Crear un nuevo objeto datetime con la hora fijada a las 04:00 del mismo día hace 7 días
        hora_ajustada = hace_7_dias.replace(hour=4, minute=0, second=0, microsecond=0)
    else:
        hora_ajustada = hace_7_dias.replace(hour=5, minute=0, second=0, microsecond=0)

    # Formatear la nueva fecha y hora en formato RFC3339 con la 'Z'
    hora_rfc3339 = hora_ajustada.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

    return hora_rfc3339

# Ejemplo de uso
hora_04_00_utc_rfc3339_hace_7_dias = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
print("Hora 04:00 UTC+0 (RFC3339) hace 7 días:", hora_04_00_utc_rfc3339_hace_7_dias)


Hora 04:00 UTC+0 (RFC3339) hace 7 días: 2024-08-11T04:00:00.000000Z


### Llamada al API y conversion a Dataframe

In [4]:
def get_candles (instrument, granularity, starting_date, count):
    session = requests.Session()
    url = f"{defs.OANDA_URL}/instruments/{instrument}/candles"
    params = {
        'granularity' : granularity,
        'price' : "MBA",
        'from' : starting_date,
        'count': count
    }
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    candle_data = response.json()
    return candle_data


def convert_to_df(candle_data):
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]
    our_data = []

    for candle in candle_data["candles"]:
        if candle["complete"] == False:
            continue
        new_dict = {}
        new_dict["time"] = candle["time"]
        new_dict["volume"] = candle["volume"]
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame(our_data)



### Logica de Iteracion para obtencion de datos consecutiva del API

In [5]:
# Intento hacer la llamada al API dentro de otra funcion que llame desde una semana atras 

hora_inicio = obtener_hora_04_00_utc_rfc3339_hace_7_dias()
instrument = "EUR_USD"
granularity = "M1"

def get_historical_candles(starting_date, instrument, granularity, count=5000, df_acumulado=None):

    json_data = get_candles(instrument, granularity, starting_date, 5000)
    print("se jalo data empezando en: ", starting_date)
    df = convert_to_df(json_data)
    last_candle = df.iloc[-1, 0]
    print ("la ultima vela es: ", last_candle)

    #condicion de cierre de recursividad
    if df.shape[0] == 1: 
        print("se cumplio la condicion de cierre del bucle recursivo")
        #df_no_duplicates = df_acumulado.drop_duplicates(subset=['time'])
        return df_acumulado.drop_duplicates(subset=['time']).reset_index(drop = True) #elimnate duplicates

    # condicion para primera iteracion
    if df_acumulado is None: 
        print("se cumplio la condicion de inicio del bucle")
        df_acumulado = df
    else:
        print("se concatenan los df")
        df_acumulado = pd.concat([df_acumulado, df])
    

    print("===================================")
    print("se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite")
    return get_historical_candles(last_candle, instrument, granularity, count=5000, df_acumulado=df_acumulado)




working_df = get_historical_candles(hora_inicio, instrument, granularity)


se jalo data empezando en:  2024-08-11T04:00:00.000000Z
la ultima vela es:  2024-08-15T09:13:00.000000000Z
se cumplio la condicion de inicio del bucle
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-08-15T09:13:00.000000000Z
la ultima vela es:  2024-08-16T20:58:00.000000000Z
se concatenan los df
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-08-16T20:58:00.000000000Z
la ultima vela es:  2024-08-16T20:58:00.000000000Z
se cumplio la condicion de cierre del bucle recursivo


## Limpieza de datos para el analisis
- [X ] Cambiar el formato de fecha a uno estandar. Mantendo el UTC+0
- [x ] Eliminar columnas innecesarias. Usare el mid price.
- [x ] Renombrar las columnas al estandar de lightweight-charts


In [6]:
hora_inicio = '2024-07-05T04:00:00.000000000Z'
instrument = "EUR_USD"
granularity = "M5"
df_to_clean =get_historical_candles(hora_inicio, instrument, granularity, count=5000, df_acumulado=None)
df_to_clean


se jalo data empezando en:  2024-07-05T04:00:00.000000000Z
la ultima vela es:  2024-07-30T12:40:00.000000000Z
se cumplio la condicion de inicio del bucle
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-07-30T12:40:00.000000000Z
la ultima vela es:  2024-08-16T20:55:00.000000000Z
se concatenan los df
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-08-16T20:55:00.000000000Z
la ultima vela es:  2024-08-16T20:55:00.000000000Z
se cumplio la condicion de cierre del bucle recursivo


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-07-05T04:00:00.000000000Z,108,1.08219,1.08219,1.08198,1.08201,1.08211,1.08211,1.08190,1.08194,1.08227,1.08227,1.08206,1.08208
1,2024-07-05T04:05:00.000000000Z,76,1.08200,1.08214,1.08200,1.08204,1.08192,1.08207,1.08192,1.08196,1.08208,1.08221,1.08208,1.08211
2,2024-07-05T04:10:00.000000000Z,76,1.08204,1.08210,1.08203,1.08206,1.08197,1.08203,1.08195,1.08199,1.08211,1.08218,1.08211,1.08214
3,2024-07-05T04:15:00.000000000Z,105,1.08206,1.08211,1.08204,1.08210,1.08197,1.08203,1.08196,1.08203,1.08214,1.08219,1.08211,1.08218
4,2024-07-05T04:20:00.000000000Z,70,1.08212,1.08213,1.08202,1.08210,1.08205,1.08206,1.08194,1.08202,1.08219,1.08220,1.08210,1.08218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8838,2024-08-16T20:35:00.000000000Z,64,1.10230,1.10236,1.10229,1.10232,1.10222,1.10228,1.10221,1.10223,1.10237,1.10244,1.10237,1.10241
8839,2024-08-16T20:40:00.000000000Z,86,1.10231,1.10244,1.10230,1.10230,1.10223,1.10236,1.10222,1.10223,1.10239,1.10251,1.10237,1.10238
8840,2024-08-16T20:45:00.000000000Z,138,1.10231,1.10266,1.10231,1.10266,1.10223,1.10257,1.10223,1.10257,1.10239,1.10274,1.10239,1.10274
8841,2024-08-16T20:50:00.000000000Z,127,1.10265,1.10297,1.10260,1.10270,1.10257,1.10288,1.10251,1.10262,1.10273,1.10307,1.10268,1.10279


In [7]:
def clean_df(df):
    df = df.rename(columns={'mid_o': 'open', 'mid_h': 'high', 'mid_l': 'low', 'mid_c': 'close'})
    df = df[['time', 'open', 'high', 'low', 'close']]
    df.set_index('time', inplace=True)
    df.index = pd.to_datetime(df.index)
    return df

cleaned_df = clean_df(df_to_clean)
cleaned_df

,open,high,low,close
time,,,,
2024-07-05 04:00:00+00:00,1.08219,1.08219,1.08198,1.08201
2024-07-05 04:05:00+00:00,1.08200,1.08214,1.08200,1.08204
2024-07-05 04:10:00+00:00,1.08204,1.08210,1.08203,1.08206
2024-07-05 04:15:00+00:00,1.08206,1.08211,1.08204,1.08210
2024-07-05 04:20:00+00:00,1.08212,1.08213,1.08202,1.08210
...,...,...,...,...
2024-08-16 20:35:00+00:00,1.10230,1.10236,1.10229,1.10232
2024-08-16 20:40:00+00:00,1.10231,1.10244,1.10230,1.10230
2024-08-16 20:45:00+00:00,1.10231,1.10266,1.10231,1.10266


In [8]:
# Slice dataframe para el horario silverbullet
start_date_1 = '2024-07-05T04:05:00.000000000Z'
finish_date_1 = '2024-07-05T04:20:00.000000000Z'

# Apply mask on function
def select_daterange(dataframe, start_date, finish_date):
    #selected = dataframe.loc[start_date:finish_date]
    selected = dataframe.loc[start_date:finish_date]
    return selected

aa = select_daterange(cleaned_df, start_date_1, finish_date_1)
aa

,open,high,low,close
time,,,,
2024-07-05 04:05:00+00:00,1.08200,1.08214,1.08200,1.08204
2024-07-05 04:10:00+00:00,1.08204,1.08210,1.08203,1.08206
2024-07-05 04:15:00+00:00,1.08206,1.08211,1.08204,1.08210
2024-07-05 04:20:00+00:00,1.08212,1.08213,1.08202,1.08210


In [9]:
cleaned_df.index = pd.to_datetime(cleaned_df.index)

In [10]:
# Prueba para saber como seleccionar solamente las candles que pertenezcan a ese rango horario.
df_silverbullet = cleaned_df.between_time('15:35:00', '23:55:00')
df_silverbullet

,open,high,low,close
time,,,,
2024-07-05 15:35:00+00:00,1.08290,1.08308,1.08282,1.08285
2024-07-05 15:40:00+00:00,1.08284,1.08290,1.08258,1.08258
2024-07-05 15:45:00+00:00,1.08259,1.08276,1.08258,1.08270
2024-07-05 15:50:00+00:00,1.08268,1.08278,1.08251,1.08274
2024-07-05 15:55:00+00:00,1.08275,1.08284,1.08269,1.08277
...,...,...,...,...
2024-08-16 20:35:00+00:00,1.10230,1.10236,1.10229,1.10232
2024-08-16 20:40:00+00:00,1.10231,1.10244,1.10230,1.10230
2024-08-16 20:45:00+00:00,1.10231,1.10266,1.10231,1.10266


## Analisis de la estrategia

#### Definiciones previas
- Inicio del dia de NY en formato Z: 04:00:00.000000000Z:
    - 2024-07-03T04:00:00.000000000Z
- Timeframe para Liquidity Grab NY 09:30-10:00 am:
    -   2024-07-03T13:30:00.000000000Z
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z
#### Parametros de la estrategia:  

- Para considerar una toma de liquidez, debo considerar la siguiente condición:

#### Criterios de la estrategia:
- Pre-Session:
    - Entre las 09:30 y 10:am NY, buscar una toma de liquidez.



In [11]:
#Constantes de la estrategia para la manana 
DAY_START = 'T04:00:00.000000000Z'
SB_AM_LOOPBACK_START = 'T13:30:00.000000000Z'
SB_AM_LOOPBACK_END = 'T13:30:00.000000000Z'
SB_AM_START = 'T13:30:00.000000000Z'
SB_AM_END = 'T13:30:00.000000000Z'


In [12]:
#def directional_change(close:np.array, high:np.array, low:np.array, sigma: float):
def directional_change(df, sigma: float):
   # conversion previa
   df_directional_change = df.reset_index()
   df_directional_change = df_directional_change[['high','low','close']]
   close = df_directional_change['close'].astype(float)
   high = df_directional_change['high'].astype(float)
   low = df_directional_change['low'].astype(float)

   up_zig = True #Last extreme is a bottom. Next is a top. Direccion del sigma
   
   # Keep track of the current index in price . ORANGE CANDLE
   temporary_max = high[0]
   temporary_min = low[0]
   temporary_max_index = 0
   temporary_min_index = 0

   tops = []
   bottoms = []

   for  current_candle_close_index in range (len(close)):
      if up_zig: # Last extreme is a bottom
         if high[current_candle_close_index] > temporary_max: 
            # New high update
            temporary_max = high[current_candle_close_index]
            temporary_max_index = current_candle_close_index
         elif close[current_candle_close_index] < temporary_max - temporary_max*sigma:
            # Price retraced by sigma%. Top confirmed, recod it
            # top[0] = confirmation index
            # top[1] = index of top
            # top[2] = price of top
            top = [current_candle_close_index, temporary_max_index, temporary_max]
            tops.append(top)

            # Setup for next bottom
            up_zig = False
            temporary_min = low[current_candle_close_index]
            temporary_min_index = current_candle_close_index
      else: # Last extreme is a top
         if low[current_candle_close_index] < temporary_min:
            # New low, update
            temporary_min = low[current_candle_close_index]
            temporary_min_index = current_candle_close_index
         elif close[current_candle_close_index] > temporary_min + temporary_min*sigma:
            # Price retraced by sigma%. Bottom confirmed, record it
            # bottom[0] = confirmation index
            # bottom[1] = index of bottom
            # bottom[2] = price of bottom
            bottom = [current_candle_close_index, temporary_min_index, temporary_min]
            bottoms.append(bottom)

            # Setup for next top
            up_zig = True
            temporary_max = high[current_candle_close_index]
            temporary_max_index = current_candle_close_index

   return tops, bottoms


In [13]:
tops = directional_change(df_silverbullet, 0.0001)[0]
bottoms = directional_change(df_silverbullet, 0.0001)[1]


In [14]:
tops

[[0, 0, 1.08308],
 [8, 7, 1.083],
 [10, 9, 1.08304],
 [19, 18, 1.08288],
 [23, 22, 1.08286],
 [26, 25, 1.08304],
 [50, 49, 1.08426],
 [53, 52, 1.08415],
 [60, 59, 1.08412],
 [69, 68, 1.08236],
 [79, 78, 1.08332],
 [85, 84, 1.08278],
 [94, 93, 1.0828],
 [100, 99, 1.08279],
 [107, 106, 1.08362],
 [119, 118, 1.08333],
 [123, 122, 1.08332],
 [129, 127, 1.08336],
 [135, 134, 1.08302],
 [143, 142, 1.08266],
 [154, 152, 1.08258],
 [161, 159, 1.08249],
 [182, 178, 1.08266],
 [202, 199, 1.08314],
 [206, 205, 1.08138],
 [208, 207, 1.08138],
 [210, 209, 1.08163],
 [223, 222, 1.08139],
 [227, 226, 1.08104],
 [229, 228, 1.08113],
 [234, 233, 1.08124],
 [244, 243, 1.08144],
 [250, 248, 1.08148],
 [254, 253, 1.0816],
 [261, 260, 1.08166],
 [276, 275, 1.0816],
 [298, 297, 1.08161],
 [304, 303, 1.08269],
 [311, 310, 1.08262],
 [315, 314, 1.08264],
 [318, 317, 1.08278],
 [322, 321, 1.08286],
 [333, 332, 1.08226],
 [344, 342, 1.08252],
 [356, 355, 1.08297],
 [363, 360, 1.08298],
 [369, 367, 1.08308],
 [3

In [15]:
bottoms

[[2, 1, 1.08258],
 [9, 8, 1.08264],
 [17, 16, 1.08233],
 [21, 20, 1.08249],
 [24, 23, 1.08254],
 [30, 28, 1.08274],
 [52, 51, 1.08394],
 [57, 55, 1.08387],
 [66, 65, 1.08044],
 [72, 71, 1.08135],
 [84, 83, 1.08243],
 [90, 89, 1.0822],
 [99, 94, 1.0826],
 [101, 100, 1.08254],
 [111, 110, 1.08285],
 [120, 119, 1.08294],
 [125, 123, 1.08304],
 [133, 132, 1.08284],
 [141, 138, 1.08235],
 [148, 147, 1.08223],
 [158, 156, 1.08234],
 [166, 165, 1.08228],
 [188, 186, 1.08243],
 [203, 202, 1.08086],
 [207, 206, 1.08105],
 [209, 208, 1.0811],
 [219, 218, 1.0806],
 [226, 225, 1.08066],
 [228, 227, 1.08086],
 [232, 231, 1.08085],
 [238, 237, 1.08082],
 [245, 244, 1.0811],
 [251, 250, 1.08124],
 [255, 254, 1.08135],
 [271, 266, 1.08137],
 [292, 289, 1.08139],
 [303, 298, 1.0814],
 [308, 307, 1.08212],
 [313, 312, 1.08238],
 [316, 315, 1.08244],
 [320, 319, 1.08254],
 [332, 331, 1.08209],
 [337, 335, 1.08174],
 [347, 344, 1.08238],
 [359, 357, 1.08274],
 [366, 365, 1.08282],
 [373, 369, 1.0828],
 [4

In [16]:
df_silverbullet

,open,high,low,close
time,,,,
2024-07-05 15:35:00+00:00,1.08290,1.08308,1.08282,1.08285
2024-07-05 15:40:00+00:00,1.08284,1.08290,1.08258,1.08258
2024-07-05 15:45:00+00:00,1.08259,1.08276,1.08258,1.08270
2024-07-05 15:50:00+00:00,1.08268,1.08278,1.08251,1.08274
2024-07-05 15:55:00+00:00,1.08275,1.08284,1.08269,1.08277
...,...,...,...,...
2024-08-16 20:35:00+00:00,1.10230,1.10236,1.10229,1.10232
2024-08-16 20:40:00+00:00,1.10231,1.10244,1.10230,1.10230
2024-08-16 20:45:00+00:00,1.10231,1.10266,1.10231,1.10266


In [17]:
df_directional_change = df_silverbullet.reset_index()
df_directional_change


,time,open,high,low,close
0,2024-07-05 15:35:00+00:00,1.08290,1.08308,1.08282,1.08285
1,2024-07-05 15:40:00+00:00,1.08284,1.08290,1.08258,1.08258
2,2024-07-05 15:45:00+00:00,1.08259,1.08276,1.08258,1.08270
3,2024-07-05 15:50:00+00:00,1.08268,1.08278,1.08251,1.08274
4,2024-07-05 15:55:00+00:00,1.08275,1.08284,1.08269,1.08277
...,...,...,...,...,...
3089,2024-08-16 20:35:00+00:00,1.10230,1.10236,1.10229,1.10232
3090,2024-08-16 20:40:00+00:00,1.10231,1.10244,1.10230,1.10230
3091,2024-08-16 20:45:00+00:00,1.10231,1.10266,1.10231,1.10266
3092,2024-08-16 20:50:00+00:00,1.10265,1.10297,1.10260,1.10270


In [18]:
df_directional_change = df_directional_change[['high','low','close']]
df_directional_change

,high,low,close
0,1.08308,1.08282,1.08285
1,1.08290,1.08258,1.08258
2,1.08276,1.08258,1.08270
3,1.08278,1.08251,1.08274
4,1.08284,1.08269,1.08277
...,...,...,...
3089,1.10236,1.10229,1.10232
3090,1.10244,1.10230,1.10230
3091,1.10266,1.10231,1.10266
3092,1.10297,1.10260,1.10270


In [19]:
closes = df_directional_change['close']

## Grafico de las velas  
Intentare solamente hacer el grafico para el dia viernes 5 julio, y el loopback en M5 para la toma de liquidez sera solo desde ese mismo dia a las 04:00 UTC+0, de acuerdo a lo que he podido recopilar de los trades de mi backtest.

In [20]:
hora_inicio = '2024-07-05T04:00:00.000000000Z'
instrument = "EUR_USD"
granularity = "M5"

df_plot_strat = get_historical_candles(hora_inicio, instrument, granularity)



se jalo data empezando en:  2024-07-05T04:00:00.000000000Z
la ultima vela es:  2024-07-30T12:40:00.000000000Z
se cumplio la condicion de inicio del bucle
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-07-30T12:40:00.000000000Z
la ultima vela es:  2024-08-16T20:55:00.000000000Z
se concatenan los df
se sigue ejecutando el bucle recursivo con el ultimo candle como nuevo limite
se jalo data empezando en:  2024-08-16T20:55:00.000000000Z
la ultima vela es:  2024-08-16T20:55:00.000000000Z
se cumplio la condicion de cierre del bucle recursivo


In [21]:

def plot_df(df):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df.index, open=df.open, high=df.high, low=df.low, close=df.close, #index puede ser time
        line=dict(width=1), opacity=1,
        increasing_fillcolor = "#24A06B",
        decreasing_fillcolor = "#CC2E3C",
        increasing_line_color = "#2EC886",
        decreasing_line_color = "#FF3A4C"
    ))
    fig.update_layout(width=1000, height=400,
                    margin=dict(l=10, r=10, b=10, t=10),
                    font = dict(size=10, color="#e1e1e1"),
                    paper_bgcolor = "#1e1e1e",
                    plot_bgcolor =  "#1e1e1e",
                    )
    fig.update_xaxes(
        gridcolor = "#1f292f",
        showgrid=True, fixedrange=True, rangeslider=dict(visible=True)
    )
    fig.update_yaxes(
        gridcolor = "#1f292f",
        showgrid=True,
    )
    return fig

plot_fig = plot_df(df_silverbullet)
plot_fig.show()

- Timeframe para inicio Liquidity Grab NY 09:30 am (horario verano):
    -   2024-07-03T13:30:00.000000000Z
- Timeframe para fin Liquidity Grab NY 10:00 am (horario verano):
    -   2024-07-03T14:00:00.000000000Z    
- Silverbullet Start NY 10:00am:
    -   2024-07-03T14:00:00.000000000Z
- Silverbullet End NY 11:00am :
    -   2024-07-03T15:00:00.000000000Z
- Fin de sesion NY 17:00:
    -   2024-07-03T21:00:00.000000000Z

In [22]:
#Constantes que debo usar para la estrategia



In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                open=df['AAPL.Open'], high=df['AAPL.High'],
                low=df['AAPL.Low'], close=df['AAPL.Close'])
                      ])



fig.update_layout(
    title='The Great Recession',
    yaxis_title='AAPL Stock',
    shapes = [dict(
        x0='2016-11-09', x1='2016-12-09', y0=-0, y1=1, xref='x', yref='paper',
        line_width=1)],
    annotations=[dict(
        x='2016-12-09', y=0.05, xref='x', yref='paper',
        showarrow=False, xanchor='right', text='Increase Period Begins')]
)

fig.show()

# UTILIDADES A NECESITAR LUEGO

In [24]:
# Concatenar varios df. usar ignore_index = False para que no reescriba los indices

import pandas as pd

# Crear cinco DataFrames de ejemplo con índices personalizados
df1 = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
}, index=['a', 'b', 'c'])

df2 = pd.DataFrame({
    'A': [7, 8, 9],
    'B': [10, 11, 12]
}, index=['d', 'e', 'f'])

df3 = pd.DataFrame({
    'A': [13, 14, 15],
    'B': [16, 17, 18]
}, index=['g', 'h', 'i'])

df4 = pd.DataFrame({
    'A': [19, 20, 21],
    'B': [22, 23, 24]
}, index=['j', 'k', 'l'])

df5 = pd.DataFrame({
    'A': [25, 26, 27],
    'B': [28, 29, 30]
}, index=['m', 'n', 'o'])

# Anexar los cinco DataFrames
df_combined = pd.concat([df1, df2, df3, df4, df5])

print(df_combined)

    A   B
a   1   4
b   2   5
c   3   6
d   7  10
e   8  11
f   9  12
g  13  16
h  14  17
i  15  18
j  19  22
k  20  23
l  21  24
m  25  28
n  26  29
o  27  30


In [25]:
#convertir a utc+0 a partir de un RFC339

fecha_prueba = "2024-06-27T04:00:00.000000000Z"
def convert_rfc3339_to_utc(fecha_prueba):
    return fecha_prueba.replace("T", " ").replace("Z", "").split(".")[0]

convert_rfc3339_to_utc(fecha_prueba)

'2024-06-27 04:00:00'

In [26]:

# Crear un DataFrame de ejemplo con un índice en formato RFC3339 
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
}
index = ['2024-07-03T04:00:00.000000000Z', '2024-07-04T05:00:00.000000000Z', '2024-07-05T06:00:00.000000000Z']
df_rfc3339 = pd.DataFrame(data, index=index)

# Eliminar la parte 'Z' del índice
df_rfc3339.index = df_rfc3339.index.str.replace('Z', '')

# Convertir el índice a tipo datetime
df_rfc3339.index = pd.to_datetime(df.index)

df

ValueError: Length mismatch: Expected axis has 3 elements, new values have 506 elements